In [237]:
import numpy as np


def diy_lu(a):
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u
        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [238]:
import numpy as np
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)
print(np.linalg.matrix_rank(a))

6


In [239]:
np.set_printoptions(precision=3)
L, u = diy_lu(a)
print(L, "\n")
print(u, "\n")
print(L @ u - a)

a1 = a.copy()
a1[1, 1] = 3 
print(np.linalg.matrix_rank(a1))

L, u = diy_lu(a1)
print(L, "\n")
print(u, "\n")
print(L @ u - a1, "\n")
#Покажу тот, который очевидно 0
print(np.linalg.det(a1[0:2,0:2]), "\n")
#А тут все не 0 были
for i in range(N):
    print(np.linalg.det(a[0:i,0:i]))

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

<ipython-input-237-413e52b068b5>:10: RuntimeWarning: divide by zero encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]
<ipython-input-237-413e52b068b5>:12: RuntimeWarning: invalid value encountered in matmul
  u = lam @ u
<ipython-input-237-413e52b068b5>:14: RuntimeWarning: invalid value encountered in matmul
  L = L @ lam
<ipython-input-237-413e52b068b5>:10: RuntimeWarning: invalid value encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]


In [240]:
def phoenix(L, U, P):
    return L @ U @ np.linalg.inv(P)
def diy_LU_v2(a):
    N = a.shape[0]
    U = a.copy()
    P_fin = np.eye(N)
    L_fin = np.zeros((N, N))
    for k in range(0, N - 1):
        max_el = k + np.argmax(abs(U[k:, k])) 
        if max_el != k:
            P = np.eye(N)
            P[[max_el, k], k:N] = P[[k, max_el], k:N]
            U[[max_el, k], k:N] = U[[k, max_el], k:N] 
            P_fin = P @ P_fin
            L_fin = P @ L_fin
        L = np.eye(N)
        for j in range(k+1, N):
            L[j, k] = -(U[j, k] / U[k, k])
            L_fin[j, k] = (U[j, k] / U[k, k])
        U = L @ U
    np.fill_diagonal(L_fin, 1)
    return P_fin, L_fin, U


np.set_printoptions(precision=3)

P, L, U = diy_LU_v2(a1)
print (P, "\n"*2, L, "\n"*2, U, "\n")
print(L @ U - P @ a1, "\n")
print (phoenix(L, U, P), "\n")
print (a1)

P, L, U = diy_LU_v2(a)
print (P, "\n"*2, L, "\n"*2, U, "\n")
print(L @ U - P @ a, "\n")
print (phoenix(L, U, P), "\n")
print (a)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]] 

 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.    1.    0.    0.    0.   ]
 [1.    0.727 0.151 1.    0.    0.   ]
 [1.    0.857 0.088 0.514 1.    0.   ]
 [1.    0.941 0.038 0.208 0.641 1.   ]] 

 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  2.220e-16  0.000e+00 -9.247e-02 -1.485e-01 -1.856e-01]
 [ 0.000e+00 -4.611e-17  0.000e+00  0.000e+00  1.841e-03  3.821e-03]
 [ 0.000e+00 -8.460e-17  2.776e-17  0.000e+00  0.000e+00 -1.233e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000